In [ ]:
import numpy as np
from multiprocessing import Pool  # , Process
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
import pandas as pd
import tensorflow as tf
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

2022-11-11 00:15:07.136414: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-11 00:15:09.410194: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (34)


In [ ]:

activation = 'relu'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 10
batch_size = 100


In [3]:

white = pd.read_csv('wine_data.csv')

tensor_x_0 =\
    tf.constant(white[['residual sugar',
                       'chlorides',
                       'total sulfur dioxide',
                       'free sulfur dioxide']].values)

tensor_x_1 =\
    tf.constant(white[['fixed acidity',
                       'volatile acidity',
                       'citric acid',
                       'density',
                       'pH',
                       'sulphates',
                       'alcohol']].values)


training_x = [tensor_x_0, tensor_x_1]
INPUT_SHAPES = [training_x[i].shape[1] for i in np.arange(len(training_x))]


#alcohol_labels = tf.constant(pd.get_dummies(pd.cut(white['alcohol'], np.arange(
#    white['alcohol'].min(), white['alcohol'].max()))).values)
quality_labels = tf.constant(pd.get_dummies(white['quality']).values)

train_labels = [quality_labels]

OUTPUT_SHAPES = [train_labels[i].shape[1]
                 for i in np.arange(len(train_labels))]


In [4]:

TIME = pendulum.now('Europe/Paris').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test'

meta_trial_number = int(np.random.random() * 10 ** 12)

cerebros = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.35,
    direction='maximize',
    metric_to_rank_by='val_top_1',
    minimum_levels=4,
    maximum_levels=7,
    minimum_units_per_level=1,
    maximum_units_per_level=4,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=4,
    activation=activation,
    final_activation='softmax',
    number_of_architecture_moities_to_try=3,
    number_of_tries_per_architecture_moity=2,
    number_of_generations=3,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss="categorical_crossentropy",
    metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1'),
             tf.keras.metrics.TopKCategoricalAccuracy(
             k=2, name='top_2'),
             tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3')],
    epochs=epochs,
    patience=7,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    use_multiprocessing_for_multiple_neural_networks=False,  # pull this param
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number)
val_top_1 = cerebros.run_random_search()


nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}, {'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'2': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'3': <class 'cerebros.units.units.DenseUnit'>}]
2
Trying to create level 2
We think level 2's predecessors are: [0, 1]
k is: 3 value is: [{'4': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}]
3
Trying to create level 3
We think level 3's predecessors are: [0, 1, 2]
k is: 4 value is: [{'3': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}]
4
Trying to create level 4
We think level 4's predecessors are: [0, 1, 2, 3]
k is: 5 value is: [{'1': <class 'cerebros.units.units.DenseU

returning trial 0 oracles
       loss     top_1     top_2     top_3  val_loss  val_top_1  val_top_2  \
0  1.999579  0.281495  0.447063  0.576186  4.526497   0.184840   0.206414   
1  1.582808  0.448633  0.688973  0.778825  3.159898   0.194169   0.324781   
2  1.399253  0.499215  0.778197  0.862080  2.322751   0.331778   0.667638   
3  1.292894  0.517436  0.817154  0.908577  1.871771   0.421574   0.683965   
4  1.214200  0.535658  0.830663  0.931511  1.567139   0.470554   0.688047   
5  1.166173  0.543198  0.839774  0.942507  1.445021   0.460641   0.714286   
6  1.134586  0.537229  0.840402  0.943135  1.326863   0.447230   0.787755   
7  1.116003  0.547597  0.838517  0.947848  1.237548   0.444898   0.808163   
8  1.100581  0.544455  0.845743  0.950047  1.196446   0.455977   0.818659   
9  1.092643  0.546654  0.843858  0.949733  1.120201   0.512536   0.844898   

   val_top_3  trial_number  subtrial_number  
0   0.688047             0                0  
1   0.688047             0        

returning trial 0 oracles
       loss     top_1     top_2     top_3  val_loss  val_top_1  val_top_2  \
0  2.113755  0.346264  0.584116  0.707636  7.220053   0.022741   0.026239   
1  1.651538  0.382344  0.638391  0.752749  4.697512   0.036735   0.062974   
2  1.475461  0.449576  0.745523  0.845115  3.403857   0.101458   0.170845   
3  1.342520  0.506126  0.799560  0.890983  2.284904   0.239650   0.387172   
4  1.249829  0.518693  0.820295  0.927113  1.732373   0.381924   0.593586   
5  1.181504  0.539428  0.833490  0.933710  1.453289   0.419825   0.755102   
6  1.144756  0.542570  0.836946  0.944706  1.293948   0.455977   0.812828   
7  1.118761  0.535344  0.840088  0.948790  1.226247   0.476385   0.832070   
8  1.098537  0.542884  0.842287  0.948476  1.163851   0.485131   0.854227   
9  1.074791  0.544769  0.851084  0.951618  1.110112   0.511370   0.866472   

   val_top_3  trial_number  subtrial_number  
0   0.043732             0                1  
1   0.130612             0        

returning trial 1 oracles
       loss     top_1     top_2     top_3  val_loss  val_top_1  val_top_2  \
0  2.035558  0.353410  0.594120  0.701511  2.705074   0.047230   0.421574   
1  1.624740  0.430097  0.649387  0.764373  2.024928   0.351020   0.501458   
2  1.455768  0.508640  0.755891  0.846686  2.050220   0.311370   0.467055   
3  1.337210  0.527176  0.804901  0.888784  1.916016   0.305539   0.479300   
4  1.252276  0.532202  0.825636  0.921144  1.758726   0.318367   0.528863   
5  1.190804  0.546968  0.834119  0.932768  1.536731   0.372595   0.644315   
6  1.149999  0.540685  0.840088  0.940622  1.387943   0.425656   0.724198   
7  1.123988  0.546968  0.842915  0.944078  1.280000   0.456560   0.777259   
8  1.100487  0.552623  0.846371  0.948476  1.187249   0.489796   0.824490   
9  1.088366  0.555765  0.845743  0.948162  1.141479   0.518367   0.837318   

   val_top_3  trial_number  subtrial_number  
0   0.556268             1                0  
1   0.672303             1        

returning trial 1 oracles
       loss     top_1     top_2     top_3  val_loss  val_top_1  val_top_2  \
0  2.093378  0.341568  0.572070  0.707023  5.094368   0.189504   0.202332   
1  1.660764  0.418159  0.638077  0.742696  5.048813   0.188921   0.201166   
2  1.474453  0.481935  0.746466  0.840088  4.146771   0.188921   0.300875   
3  1.353314  0.515866  0.789821  0.890669  3.056567   0.194169   0.551020   
4  1.255625  0.525919  0.819981  0.924285  2.216079   0.251895   0.626822   
5  1.191539  0.524034  0.830349  0.941250  1.772107   0.313120   0.738192   
6  1.152705  0.535344  0.835375  0.947220  1.435332   0.397085   0.797668   
7  1.123475  0.544769  0.842287  0.952875  1.260552   0.434402   0.816327   
8  1.109806  0.541942  0.840716  0.950361  1.178871   0.461808   0.835568   
9  1.088105  0.548539  0.846686  0.953817  1.117088   0.492711   0.847813   

   val_top_3  trial_number  subtrial_number  
0   0.381341             1                1  
1   0.226239             1        

returning trial 2 oracles
       loss     top_1     top_2     top_3  val_loss  val_top_1  val_top_2  \
0  2.142703  0.323193  0.549816  0.677624  2.702482   0.253644   0.560933   
1  1.772225  0.356896  0.566133  0.672950  2.522099   0.293294   0.389504   
2  1.582973  0.432297  0.673578  0.772227  2.391143   0.226239   0.317784   
3  1.449098  0.468740  0.754948  0.848256  2.148192   0.256560   0.344023   
4  1.336649  0.496387  0.798618  0.896953  1.762921   0.299125   0.494461   
5  1.261700  0.511153  0.813384  0.930883  1.517880   0.348105   0.661224   
6  1.213471  0.516494  0.817468  0.944078  1.381150   0.382507   0.735860   
7  1.173493  0.521835  0.821866  0.947220  1.284918   0.411662   0.794752   
8  1.150917  0.528746  0.827521  0.948476  1.187283   0.475802   0.840816   
9  1.136030  0.519950  0.832862  0.953817  1.163265   0.489213   0.851312   

   val_top_3  trial_number  subtrial_number  
0   0.690962             2                0  
1   0.482799             2        

returning trial 2 oracles
       loss     top_1     top_2     top_3  val_loss  val_top_1  val_top_2  \
0  2.154047  0.331360  0.567579  0.689873  3.706240   0.007580   0.051312   
1  1.704719  0.410305  0.622369  0.733585  2.695668   0.033819   0.131778   
2  1.510797  0.481621  0.729186  0.830349  2.360175   0.129446   0.288630   
3  1.408590  0.510211  0.770342  0.872762  2.135693   0.245481   0.440233   
4  1.309140  0.525919  0.800503  0.907320  1.904387   0.278717   0.528280   
5  1.251422  0.523406  0.816211  0.927113  1.681699   0.334111   0.630904   
6  1.204209  0.530946  0.826579  0.934653  1.511303   0.369096   0.716618   
7  1.168370  0.535344  0.822180  0.942507  1.353194   0.398834   0.780758   
8  1.140168  0.537229  0.832862  0.947220  1.260607   0.411662   0.815743   
9  1.127713  0.540057  0.836318  0.949419  1.206041   0.433236   0.840233   

   val_top_3  trial_number  subtrial_number  
0   0.364432             2                1  
1   0.360933             2        

In [5]:
print(val_top_1)

0.518367350101471
